<a href="https://colab.research.google.com/github/CSID-DGU/2021-1-OSSP1-Debugger-4/blob/autoencoder/Training_K_Face_model_with_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리 설정
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import cv2
import gc
import glob
from tensorflow.keras.layers import Reshape
from keras.models import load_model
import keras

# 랜덤 시드 고정
SEED=2021
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
import random
load_y=np.load('../Upscaling/Dataset_output/NON-WEAR_dataset.npz')  #이미지 데이터가 저장된 NPZ 로드하기.
Y_1 =load_y['arr_0']
load_y.close()
#np.random.shuffle(Y_1)

  
Y_test = Y_1[-1000:] 
Y_1 = Y_1[:-1000]

del load_y
print(Y_1.shape)
gc.collect()

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1","/gpu:3","/gpu:4","/gpu:5","/gpu:6","/gpu:7","/gpu:2"])

8 Physical GPUs, 8 Logical GPUs
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7', '/job:localhost/replica:0/task:0/device:GPU:2')


In [ ]:

# 오토인코더 모델 정의
def Autoencoder():
  

  with strategy.scope():
      X = tf.keras.layers.Input(shape=[800,800,3])

  #Encoder (Convolution layer 4겹)
      H = tf.keras.layers.Conv2D(4,(4,4),(2,2),activation='relu',padding='same')(X)
      H = tf.keras.layers.Conv2D(4*2,(4,4), (2,2),activation='relu',padding='same')(H)
      H = tf.keras.layers.Conv2D(4*4, (4,4),(2,2),activation='relu',padding='same')(H)
      H = tf.keras.layers.Conv2D(4*8, (4,4),(2,2),activation='relu',padding='same')(H)
      H = tf.keras.layers.Conv2D(4*16, (4,4),(2,2),activation='relu',padding='same')(H)
      H = tf.keras.layers.Conv2D(4*32, (4,4),(2,2),activation='relu',padding='same')(H)

 

      pool_tmp = H.shape.as_list()
      pool_tmp=pool_tmp[1:]
  
  #Fully-Connected
      H = tf.keras.layers.Flatten()(H)
      flatten_tmp=H.shape.as_list()
      flatten_tmp=flatten_tmp[1]
 
      H = tf.keras.layers.Dense(10000,activation='relu')(H)
      H = tf.keras.layers.Dense(flatten_tmp,activation='relu')(H)
      H = Reshape(pool_tmp)(H)


  #Decoder

      H = tf.keras.layers.Conv2DTranspose(4*16,(2,2),strides=(2,2),padding='same',activation='relu')(H)
      H = tf.keras.layers.Conv2DTranspose(4*8,(2,2),strides=(2,2),padding='same',activation='relu')(H)
      H = tf.keras.layers.Conv2DTranspose(4*4,(2,2),strides=(2,2),padding='same',activation='relu')(H)
      H = tf.keras.layers.Conv2DTranspose(4*2,(2,2),strides=(2,2),padding='same',activation='relu')(H)
      H = tf.keras.layers.Conv2DTranspose(4,(2,2),strides=(2,2),padding='same',activation='relu')(H)
      H = tf.keras.layers.Conv2DTranspose(3,(2,2),strides=(2,2),padding='same',activation='relu')(H)
  

  
  
      model = tf.keras.models.Model(inputs=X, outputs=H)
      model.compile(optimizer=keras.optimizers.Adam(0.01*8),loss='mean_squared_logarithmic_error', metrics=['acc']) #MSLE Loss Funciton 적용
      return model

ae_model = Autoencoder()
ae_model.summary()
#tf.compat.v1.disable_eager_execution()



INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 800, 800, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 400, 400, 4)       196       
_________________________________________________________________
conv2d_1 (Conv2D)            

In [ ]:
#모델 체크포인트 설정
checkpoint_path = './check/checkpoint.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only = True,
                                                save_best_only = True,
                                                monitor='val_acc',
                                                verbose=1)

In [ ]:

history = ae_model.fit(Y_1,
                       Y_1,
                       batch_size=512, #한번에 학습할 데이터 Batch_size
                       epochs=200,      #에포크 설정
                       verbose=1,      #학습과정 시각화     
                       shuffle=True,
                       callbacks=[checkpoint],  #모델 체크포인트 저장
                       validation_split=0.2)         